In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
  #  for filename in filenames:
    #    print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import cv2
import itertools
import csv
import tensorflow as tf
import keras
from collections import defaultdict
from keras.models import *
from keras.layers import *
from keras import optimizers
from keras.utils import plot_model

In [3]:
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)

In [4]:
def getInputArr( path ,path1 ,path2 , width , height):
	try:
        #read the image
		img = cv2.imread(path, 1)
		#resize it 
		img = cv2.resize(img, ( width , height ))
		#input must be float type
		img = img.astype(np.float32)

		#read the image
		img1 = cv2.imread(path1, 1)
        #resize it 
		img1 = cv2.resize(img1, ( width , height ))
		#input must be float type
		img1 = img1.astype(np.float32)

		#read the image
		img2 = cv2.imread(path2, 1)
		#resize it 
		img2 = cv2.resize(img2, ( width , height ))
		#input must be float type
		img2 = img2.astype(np.float32)

        #combine three imgs to  (width , height, rgb*3)
		imgs =  np.concatenate((img, img1, img2),axis=2)

		#since the odering of TrackNet  is 'channels_first', so we need to change the axis
		imgs = np.rollaxis(imgs, 2, 0)
		return imgs

	except Exception as e:

		print(path+str(e))



#get output array
def getOutputArr( path , nClasses ,  width , height  ):

	seg_labels = np.zeros((  height , width  , nClasses ))
	try:
		img = cv2.imread(path, 1)
		img = cv2.resize(img, ( width , height ))
		img = img[:, : , 0]

		for c in range(nClasses):
			seg_labels[: , : , c ] = (img == c ).astype(int)

	except Exception as e:
		print(e)
		
	seg_labels = np.reshape(seg_labels, ( width*height , nClasses ))
	return seg_labels



#read input data and output data
def InputOutputGenerator( images_path,  batch_size,  n_classes , input_height , input_width , output_height , output_width ):


	#read csv file to 'zipped'
	columns = defaultdict(list)
	with open(images_path, 'r') as f:
	    reader = csv.reader(f)
	    next(reader)
	    for row in reader:
	        for (i,v) in enumerate(row):
	            columns[i].append(v)
	zipped = itertools.cycle( zip(columns[0], columns[1], columns[2], columns[3]) )

	while True:
		Input = []
		Output = []
		#read input&output for each batch
		for _ in range( batch_size) :
			path, path1, path2 , anno = next(zipped) 
			Input.append( getInputArr(path, path1, path2 , input_width , input_height) )
			Output.append( getOutputArr( anno , n_classes , output_width , output_height) )
		#return input&output
		yield np.array(Input) , np.array(Output)

In [5]:
def SegNet(input_shape=(9,360, 640), classes=256):
    img_input = Input(shape=input_shape)
    x = img_input
    # Encoder
    x = Conv2D(64, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(64, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2,2), data_format='channels_first', padding="same")(x)

    x = Conv2D(128, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(128, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2,2), data_format='channels_first', padding="same")(x)

    x = Conv2D(256, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(256, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(256, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = MaxPooling2D(pool_size=(2,2), data_format='channels_first', padding="same")(x)

    x = Conv2D(512, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(512, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(512, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    
    # Decoder
    
    x = Conv2D(512, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(512, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(512, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling2D(size=(2, 2),  data_format='channels_first')(x)
    x = Conv2D(256, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(256, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(128, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = UpSampling2D(size=(2, 2),  data_format='channels_first')(x)
    x = Conv2D(128, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(64, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = UpSampling2D(size=(2, 2),  data_format='channels_first')(x)
    x = Conv2D(64, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(classes, 3, padding="same", kernel_initializer='random_uniform', data_format='channels_first')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    #x = Conv2D(classes, 1, 1, padding="valid")(x)
    o_shape = Model(img_input, x).output_shape
    print("output shape:", o_shape[1], o_shape[2], o_shape[3])
    # layer24 output shape: 256, 360, 640

    OutputHeight = o_shape[2]
    OutputWidth = o_shape[3]

    # reshape the size to (256, 360*640)
    x = (Reshape((-1, OutputHeight * OutputWidth)))(x)

    # change dimension order to (360*640, 256)
    x = (keras.layers.Permute((2, 1)))(x)

    gaussian_output = (Activation('softmax'))(x)

    model = Model(img_input, gaussian_output)
    model.outputWidth = OutputWidth
    model.outputHeight = OutputHeight

    # show model's details
    model.summary()

    return model

In [6]:
training_images_name = "../input/cip-project-dataset/CIP_project/train_model_3.csv"
train_batch_size = 2
n_classes = 256
input_height = 360
input_width = 640
save_weights_path = "segnet_weights_5"
epochs = 40
step_per_epochs = 200
optimizer_name = optimizers.Adadelta(lr=1.0)

In [7]:
#modelTN = TrackNet( n_classes ,  input_height, input_width )
model = SegNet()
model.compile(loss='categorical_crossentropy', optimizer= optimizer_name, metrics=['accuracy'])
model.load_weights('../input/segnet7/segnet_weights_5.0')

output shape: 256 360 640
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 9, 360, 640)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 64, 360, 640)      5248      
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 360, 640)      2560      
_________________________________________________________________
activation (Activation)      (None, 64, 360, 640)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 360, 640)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 360, 640)      2560      
_________________________________________________________________
activation_1 (Activation)    (Non

In [8]:
model_output_height = 360
model_output_width = 640

In [9]:
rator = InputOutputGenerator( training_images_name,  train_batch_size,  n_classes , input_height , input_width , model_output_height , model_output_width)

In [10]:
model.fit( rator, epochs=epochs,steps_per_epoch=step_per_epochs, verbose=1)

Epoch 1/40
200/200 [==============================] - 720s 4s/step - loss: 0.0017 - accuracy: 0.9996
Epoch 2/40
200/200 [==============================] - 718s 4s/step - loss: 0.0017 - accuracy: 0.9996
Epoch 3/40
200/200 [==============================] - 723s 4s/step - loss: 0.0016 - accuracy: 0.9996
Epoch 4/40
200/200 [==============================] - 728s 4s/step - loss: 0.0016 - accuracy: 0.9996
Epoch 5/40
200/200 [==============================] - 726s 4s/step - loss: 0.0017 - accuracy: 0.9996
Epoch 6/40
200/200 [==============================] - 733s 4s/step - loss: 0.0017 - accuracy: 0.9996
Epoch 7/40
200/200 [==============================] - 734s 4s/step - loss: 0.0016 - accuracy: 0.9996
Epoch 8/40
200/200 [==============================] - 738s 4s/step - loss: 0.0016 - accuracy: 0.9996
Epoch 9/40
200/200 [==============================] - 732s 4s/step - loss: 0.0017 - accuracy: 0.9996
Epoch 10/40
200/200 [==============================] - 744s 4s/step - loss: 0.0016 - accura

In [11]:
model.save_weights(save_weights_path+".0")

In [12]:
model.save("segnet5_mod.h5")